In [1]:
import numpy as np
from montecolor.distance import CVD_TYPES
from montecolor.palette import Palette

# Working with palettes

Each palette includes white as a fixed color by default. Other colors default to black.

In [2]:
p = Palette(4)
p.variable_colors

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

When run, the following cell will display the actual colors as squares.

In [3]:
p.to_seaborn()

[array([1., 1., 1.]),
 array([0., 0., 0.]),
 array([0., 0., 0.]),
 array([0., 0., 0.])]

This won't persist when reopening a saved notebook, but you can always redisplay them using the following:
```
from seaborn.palettes import _ColorPalette
_ColorPalette(p.variable_colors/255)
```

We can set the non-fixed colors as follows:

In [4]:
p.set_variable_colors([
    [255, 0, 0],
    [0, 255, 0],
    [0, 0, 255]
])
p.to_seaborn()

[array([1., 1., 1.]),
 array([1., 0., 0.]),
 array([0., 1., 0.]),
 array([0., 0., 1.])]

# Setting up CVD weights

We will limit ourselves to single types of color vision deficiency by setting only the first row to be nonzero in CVD-space. We'll give double weight to non-CVD distance, since this is much more common in the population. We'll also give lower weight to tritanomaly.

In [5]:
cvd_matrix = np.zeros((len(CVD_TYPES), len(CVD_TYPES)))
cvd_matrix[0, 0] = 1  # No CVD
cvd_matrix[0, 1] = 0.5  # deuteranomaly
cvd_matrix[0, 2] = 0.5  # protanomaly
cvd_matrix[0, 3] = 0.25  # tritanomaly (rarer form of CVD)

# Generating a three-color palette

In [6]:
%%time
p = Palette(4, cvd_matrix=cvd_matrix)
p.generate()

100%|█████████████████████████████████████████| 100/100 [00:39<00:00,  2.52it/s]

CPU times: user 39.6 s, sys: 1.94 s, total: 41.5 s
Wall time: 40.1 s


np.float64(0.02281758825387751)

The output is the cost associated with this color palette.

In [7]:
p.variable_colors

array([[ 83.09387207,  76.64025879,  15.54077148],
       [252.44567871, 216.66772461,   9.89099121],
       [ 30.20632935,  17.33544922, 146.18786621]])

In [8]:
p.to_seaborn()

[array([1., 1., 1.]),
 array([0.32585832, 0.30055003, 0.0609442 ]),
 array([0.98998305, 0.84967735, 0.0387882 ]),
 array([0.11845619, 0.06798215, 0.57328575])]

If visibility against the white background (index `0`) is a priority, set `pair_matrix` to upweight the cost of pairings with that fixed color:

In [9]:
pair_matrix = np.ones((3, 4))
pair_matrix[:, 0] = 10
p = Palette(4, cvd_matrix=cvd_matrix, pair_matrix=pair_matrix)
p.generate()

100%|█████████████████████████████████████████| 100/100 [00:39<00:00,  2.54it/s]


np.float64(0.3128358587087169)

In [10]:
p.variable_colors

array([[  8.46915574,  10.60350638,  43.77535077],
       [235.22507337,  18.92498456, 219.5296373 ],
       [ 52.87284179, 126.92778933,  15.14357708]])

In [11]:
p.to_seaborn()

[array([1., 1., 1.]),
 array([0.03321238, 0.04158238, 0.17166804]),
 array([0.92245127, 0.07421563, 0.86090054]),
 array([0.20734448, 0.49775604, 0.05938658])]

# Generating an eight-color palette

In [12]:
p = Palette(9, cvd_matrix=cvd_matrix)
p.generate()
p.to_seaborn()

100%|█████████████████████████████████████████| 100/100 [12:26<00:00,  7.47s/it]


[array([1., 1., 1.]),
 array([0.30788377, 0.84383796, 0.70603751]),
 array([0.13674759, 0.0061314 , 0.09395566]),
 array([0.13258817, 0.1186831 , 0.60513734]),
 array([0.78538156, 0.40156337, 0.60155855]),
 array([0.95078101, 0.93817785, 0.20244289]),
 array([0.60167292, 0.32246797, 0.11446611]),
 array([0.1475515 , 0.46325983, 0.28626329]),
 array([0.4980626 , 0.36607349, 0.8562561 ])]

In [13]:
p.variable_colors

array([[ 78.51036072, 215.17868042, 180.03956604],
       [ 34.87063599,   1.56350708,  23.95869446],
       [ 33.8099823 ,  30.26419067, 154.3100214 ],
       [200.27229691, 102.39865875, 153.39743042],
       [242.44915771, 239.23535156,  51.62293625],
       [153.42659473,  82.22933197,  29.18885803],
       [ 37.62563324, 118.1312561 ,  72.99713898],
       [127.00596237,  93.34873962, 218.3453064 ]])

In [2]:
from montecolor.distance import distance_matrix, cost_matrix, weighted_cost

# Analyze specific color pairs
red = [255, 0, 0]
green = [0, 255, 0]

# Get distances across all CVD combinations
distances = distance_matrix(red, green)
print(f"Distance matrix shape: {distances.shape}")  # (4, 4, 1)

# Convert to optimization costs
costs = cost_matrix(distances)

# Get weighted average cost
avg_cost = weighted_cost(red, green)
print(f"Average discrimination cost: {avg_cost:.3f}")

Distance matrix shape: (4, 4)
Average discrimination cost: 0.006
